# Demonstration der Bibliothek OSMnx

[OSMnx](https://github.com/gboeing/osmnx) ist eine Python-Bibliothek, über welche sich georeferenzierte Daten von dem [OpenStreetMap](https://www.openstreetmap.org)-Server abrufen lassen.

In [ ]:
import osmnx as ox
import networkx as nx
from IPython.display import Image

# turn response caching off
ox.config(use_cache=False)

Umrisse einer Stadt oder einer Region anzeigen lassen:

In [ ]:
# get the boundary polygon for a city, project it, and plot it
city = ox.geocode_to_gdf("Berlin, Germany")
#city = ox.geocode_to_gdf("Berkeley, CA")
city_proj = ox.project_gdf(city)
ax = city_proj.plot(fc="grey", ec="none")
_ = ax.axis("off")

Straßennetz im Umkreis eines Orts anzeigen:

In [ ]:
# define a point at the Rooftop-building in Berlin
location_rooftop = (52.5166094,13.3227239)

# create drive network from point, inside bounding box of N, S, E, W each 1000m from point
G = ox.graph_from_point(location_rooftop, dist=1000, dist_type="bbox", network_type="drive")
fig, ax = ox.plot_graph(G, node_color="r")

Fuß-Wegenetz im Umkreis eines Orts anzeigen:

In [ ]:
# create walk network from point, inside bounding box of N, S, E, W each 1000m from point
G = ox.graph_from_point(location_rooftop, dist=1000, dist_type="bbox", network_type="walk")
fig, ax = ox.plot_graph(G, node_color="r")

Wegenetz einfärben

In [ ]:
# convert graph to line graph so edges become nodes and vice versa
edge_centrality = nx.closeness_centrality(nx.line_graph(G))
nx.set_edge_attributes(G, edge_centrality, "edge_centrality")

# color edges in original graph with closeness centralities from line graph
ec = ox.plot.get_edge_colors_by_attr(G, "edge_centrality", cmap="inferno")
fig, ax = ox.plot_graph(G, edge_color=ec, edge_linewidth=2, node_size=0)

Eine Rote auf der Straße finden

In [ ]:
# impute missing edge speeds and calculate edge travel times with the speed module
#G = ox.graph_from_point(location_rooftop, dist=1000, dist_type="bbox", network_type="walk")
G = ox.graph_from_point(location_rooftop, dist=1000, dist_type="bbox", network_type="drive")
G = ox.speed.add_edge_speeds(G)
G = ox.speed.add_edge_travel_times(G)
# get the nearest network nodes to two lat/lng points with the distance module
# location Rooftop building
orig = ox.distance.nearest_nodes(G, X=location_rooftop[1], Y=location_rooftop[0])
location_HA33 = (52.5089999,13.3266115)
# location Hardenbergstrasse 33
dest = ox.distance.nearest_nodes(G, X=location_HA33[1], Y=location_HA33[0])
# find the shortest path between nodes, minimizing travel time, then plot it
route = ox.shortest_path(G, orig, dest, weight="travel_time")
fig, ax = ox.plot_graph_route(G, route, node_size=0)

Gebäusegrundrisse anzeigen und Flächen berechnen

In [ ]:
# configure the inline image display
img_folder = "images"
extension = "png"
size = 240
# specify that we're retrieving building footprint geometries
tags = {"building": True}

dist = 65
point = location_rooftop
gdf = ox.geometries_from_point(point, tags, dist=dist)

gdf_proj = ox.project_gdf(gdf)
bbox = ox.utils_geo.bbox_from_point(point=point, dist=dist, project_utm=True)
fp = f"./{img_folder}/rooftop_bldgs.{extension}"
fig, ax = ox.plot_footprints(
    gdf_proj,
    bbox=bbox,
    color="w",
    filepath=fp,
    dpi=90,
    save=True,
    show=False,
    close=True,
)
Image(fp, height=size, width=size)

In [ ]:
# calculate the area in projected units (meters) of each building footprint, then display first five
areas = gdf_proj.area
areas.head(6)

In [ ]:
# total area (m2) covered by building footprints
sum(areas)

Straßennetzwerk und Gebäudegrundrisse anzeigen

In [ ]:
# street network and building footprints
img_folder = "images"
place = "streets_and_buildings"
extension = "png"
size = 240

dist = 200
point = location_rooftop
tags = {"building": True}
network_type="drive"
dpi=40

fp = f"./{img_folder}/{place}.{extension}"

gdf = ox.geometries_from_point(point, tags, dist=dist)
fig, ax = ox.plot_figure_ground(
    point=point,
    dist=dist,
    network_type=network_type,
    default_width=4,
    street_widths=None,
    save=False,
    show=False,
    close=True,
)
fig, ax = ox.plot_footprints(
    gdf, 
    ax=ax, 
    filepath=fp, 
    dpi=dpi, 
    save=True, 
    show=False, 
    close=True)
Image(fp, height=size, width=size)

OSM-Karte mit Gebäuden (Zufallsstandort in Deutschland) anzeigen lassen

In [ ]:
import random

# turn response caching off
ox.config(use_cache=False)
import folium
from IPython.display import Image
from geopy.geocoders import Nominatim

# Dimension of the area of the location
dist = 1000
maxLat = 54.91159734796764 # -90 to 90, e.g. Germany: 54.91159734796764
minLat = 47.27294489442995 # -90 to 90, e.g. Germany: 47.27294489442995
maxLon = 15.041531189221159 # -180 to 180, e.g. Germany: 15.041531189221159
minLon = 5.866688951881884 # -180 to 180, e.g. Germany: 5.866688951881884
loc =(random.random()*(maxLat-minLat)+minLat,random.random()*(maxLon-minLon)+minLon)
#loc = (52.5166094,13.3227239) # Einsteinufer 43-53, 10587 Berlin
print(loc)

Geografische Informationen zu dem Ort anzeigen

In [ ]:
# initialize Nominatim API 
geolocator = Nominatim(user_agent="geoapiExercises")

# Latitude & Longitude input
Latitude = str(loc[0])
Longitude = str(loc[1])
  
location = geolocator.reverse(Latitude+","+Longitude)

# Display
print('Ort:', location,'\n')

address = location.raw['address']
print(address,'\n')

country = address.get('country', '')
state = address.get('state', '')
county = address.get('county', '')
city = address.get('city', '')
municipality = address.get('municipality', '')
neighbourhood = address.get('neighbourhood', '')
suburb = address.get('suburb', '')
borough = address.get('borough', '')
village = address.get('village', '')
zipcode = address.get('postcode')
road = address.get('road')
house_number = address.get('house_number')

print('Country:', country)
print('State:', state)
print('County:', county)
print('City:', city)
print('Municipality:', municipality)
print('Borough:', borough)
print('Suburb:', suburb)
print('Neighbourhood:', neighbourhood)
print('Village:', village)
print('Zip Code:', zipcode)
print('Road:', road)
print('House_number:', house_number)

In [ ]:
tags = {"building": True}
building = ox.geometries_from_point(loc, tags, dist)
buildings = building[building.geom_type == 'Polygon'][:2000]
m = folium.Map(loc, zoom_start=15)
folium.GeoJson(buildings[:1000]).add_to(m)
m